In [9]:
#Descomentar si se necesita instalar alguna de las librerias
import glob
import pickle
import sys
#!{sys.executable} -m pip install numpy
#!{sys.executable} -m pip install pandas
#!{sys.executable} -m pip install scikit-learn

In [3]:
import numpy as np
import pandas as pd

In [26]:
from constants import POS_TAGGED_FOLDER, SPACY_FOLDER
sys.path.append(SPACY_FOLDER)  
from feature_extractors import automatic_feature_extractor

In [135]:
def get_tagged_sentences(folder):
    # Load all the tagged sentences included in the .pickle files 
    parsed_sentences = []
    for filename in glob.glob(folder + '*.pickle'):
        with open(filename, 'rb') as tagged_file:
            parsed_sentences = parsed_sentences + pickle.load(tagged_file, encoding="bytes")
    return parsed_sentences

In [136]:
tagged_sentences = get_tagged_sentences(POS_TAGGED_FOLDER)

In [139]:
tagged_sentences[0][b'classification'].decode('utf8')

'non-fact-checkable'

In [138]:
tagged_sentences[30][b'sentence']

'Durante al año 2014 numerosos dirigentes internacionales presidentes organismos multilaterales de crédito y organismos multilaterales en general auguraban que el 2014 iba a ser un año donde culminara terminara la crisis iniciada por la caída de Lehman Brothers en el 2008'

In [22]:
print(tagged_sentences[0]['pos_tag'][3]['lemma'])

presidenta


In [23]:
tagged_sentences[0]['pos_tag'][3].keys()

dict_keys(['dep', 'text', 'pos', 'lemma', 'tag', 'like_num', 'is_punct', 'tense'])

In [27]:
dataset = [(automatic_feature_extractor(sent['pos_tag'], pos_ngrams=True), sent['classification']) for sent in tagged_sentences]

In [28]:
dataset[0]

({'PUNCT': True,
  '\ufeffdiscurso': True,
  'ROOT': True,
  'undefined': True,
  'ADP': True,
  'de': True,
  'case': True,
  'DET': True,
  'la': True,
  'det': True,
  'PROPN': True,
  'presidenta': True,
  'nmod': True,
  'cristina': True,
  'appos': True,
  'fernández': True,
  'flat': True,
  'kirchner': True,
  'en': True,
  'inauguracion': True,
  'del': True,
  'NUM': True,
  '133': True,
  'nummod': True,
  '3_digits': True,
  '°': True,
  'periodo': True,
  'NOUN': True,
  'sesiones': True,
  'ordinarias': True,
  'congreso': True,
  'nacional': True,
  'amod': True,
  ('PUNCT', 'ADP', 'DET'): True,
  ('ADP', 'DET', 'PROPN'): True,
  ('DET', 'PROPN', 'PROPN'): True,
  ('PROPN', 'PROPN', 'PROPN'): True,
  ('PROPN', 'PROPN', 'ADP'): True,
  ('PROPN', 'ADP', 'PROPN'): True,
  ('ADP', 'PROPN', 'ADP'): True,
  ('PROPN', 'ADP', 'DET'): True,
  ('DET', 'PROPN', 'ADP'): True,
  ('PROPN', 'ADP', 'NUM'): True,
  ('ADP', 'NUM', 'PROPN'): True,
  ('NUM', 'PROPN', 'PROPN'): True,
  ('PRO

In [71]:
from spacy.lang.es.stop_words import STOP_WORDS
from spacy.lang.es import Spanish

stop_words = STOP_WORDS

In [72]:
parser = Spanish()

In [170]:
def pos_tag(sentence):
    mytokens = parser(sentence)
    tags = []
    for token in mytokens:
        
        pos = token.pos_
        lemma = token.lemma_.lower()
        #word['text'] = token.text
        #word['tag'] = token.tag_
        #word['dep'] = token.dep_
        #is_punct = token.is_punct
        #like_num = token.like_num
        
        if "Tense" in token.tag_:
            trunk = token.tag_[token.tag_.find("Tense") :]
            trunk = trunk[trunk.find('=')+1:trunk.find('|')]
            tense = trunk
        else:
            tense = "undefined"

        tags.extend([pos, lemma, tense])

    return tags

In [164]:
import string
punctuations = string.punctuation

# Creating our tokenizer function
def spacy_tokenizer(sentence):
    # Creating our token object, which is used to create documents with linguistic annotations.
    mytokens = parser(sentence)

    # Lemmatizing each token and converting each token into lowercase
    mytokens = [ word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in mytokens ]

    # Removing stop words
    mytokens = [ word for word in mytokens if word not in stop_words and word not in punctuations ]
    import pdb; pdb.set_trace()
    
    pos_tokens = [ word.token.pos_ for word in mytokens ]
    word['like_num'] = token.like_num
    if "Tense" in token.tag_:
            trunk = token.tag_[token.tag_.find("Tense") :]
            trunk = trunk[trunk.find('=')+1:trunk.find('|')]
            word['tense'] = trunk
        else:
            word['tense'] = "undefined"

    # return preprocessed list of tokens
    return mytokens

In [63]:
from sklearn.base import TransformerMixin

# Custom transformer using spaCy
class predictors(TransformerMixin):
    def transform(self, X, **transform_params):
        # Cleaning Text
        return [clean_text(text) for text in X]

    def fit(self, X, y=None, **fit_params):
        return self

    def get_params(self, deep=True):
        return {}

# Basic function to clean the text
def clean_text(text):
    # Removing spaces and converting text into lowercase
    return text.strip().lower()

In [33]:
from sklearn.feature_extraction.text import CountVectorizer

In [123]:
bow_vector = CountVectorizer(tokenizer = spacy_tokenizer, ngram_range=(1,3))

In [153]:
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer

In [36]:
tfidf_vector = TfidfVectorizer(tokenizer = spacy_tokenizer)

In [154]:
tfidf_transformer = TfidfTransformer()

In [141]:
data = [{'sentence': item[b'sentence'].decode('utf8').lower(), 'target': item[b'classification'].decode('utf8')} for item in tagged_sentences]

In [42]:
df = pd.DataFrame(data)

In [43]:
from sklearn.model_selection import StratifiedShuffleSplit

split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in split.split(df, df['target']):
    strat_train_set = df.loc[train_index]
    strat_test_set = df.loc[test_index]

In [44]:
strat_train_set.target.value_counts()

non-fact-checkable    1914
fact-checkable        1175
Name: target, dtype: int64

In [45]:
strat_test_set.target.value_counts()

non-fact-checkable    479
fact-checkable        294
Name: target, dtype: int64

In [46]:
X_train = strat_train_set.drop(['target'], axis=1)
X_test = strat_test_set.drop(['target'], axis=1)
y_train = strat_train_set['target']
y_test = strat_test_set['target']

In [101]:
from sklearn.model_selection import train_test_split

In [142]:
X = [e['sentence'] for e in data]
y = [e['target'] for e in data]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
X_train = strat_train_set.drop(['target'], axis=1)
X_test = strat_test_set.drop(['target'], axis=1)
y_train = strat_train_set['target']
y_test = strat_test_set['target']

In [143]:
X_train

['uno de cada tres empleos se genera en forma directa o indirecta por el campo argentino',
 'pero lo cierto es que este acuerdo tan malo no debe ser cuando todavía irán no ha cumplimentado la parte que debía hacer',
 'nos sigue méxico con 17% perú con 14 brasil con 12 colombia con 11 y chile con 10',
 'entonces para ese el nivel de desafío tenemos que tener la flexibilidad de introducir permanentemente mejoras',
 'hicimos una primera industrialización ya no exportamos porotos de soja exportamos aceite',
 'pero yo les estaba diciendo el problema que es también el desarrollo imperfecto de nuestra sustitución de importaciones lo que no es por imperfecciones nuestras sino por cómo se distribuye la economía mundial y que está volviendo ahora a tener lugar a partir de la recuperación de las economías desarrolladas de la crisis de 2008 donde van a tender a que los resultados de las crisis que ellos provocaron las paguemos las economías emergentes',
 'dejamos el gobierno con 59% de desocupació

In [165]:
count_vect = CountVectorizer(tokenizer = spacy_tokenizer, ngram_range=(1,3))
X_train_counts = count_vect.fit_transform(X_train)

> <ipython-input-164-5e2a81e789a1>(17)spacy_tokenizer()
     13     mytokens = [ word for word in mytokens if word not in stop_words and word not in punctuations ]
     14     import pdb; pdb.set_trace()
     15 
     16     # return preprocessed list of tokens
---> 17     return mytokens

ipdb> mytokens
['empleos', 'genera', 'forma', 'directa', 'o', 'indirecta', 'campo', 'argentino']
ipdb> word
*** NameError: name 'word' is not defined
ipdb> sentence
'uno de cada tres empleos se genera en forma directa o indirecta por el campo argentino'
ipdb> q


BdbQuit: 

In [171]:
count_vect = CountVectorizer(tokenizer = pos_tag, ngram_range=(1,3))
X_train_counts = count_vect.fit_transform(X_train)

In [172]:
count_vect.vocabulary_.get(u'pobreza')

31409

In [173]:
count_vect.vocabulary_.get(u'inflación')

26885

In [175]:
# Logistic Regression Classifier
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics


classifier = LogisticRegression()

nb = MultinomialNB()

pipe = Pipeline([("cleaner", predictors()),
                 ('vectorizer', count_vect),
                 ('classifier', classifier)])


# model generation
pipe.fit(X_train,y_train)

# Predicting with a test dataset
predicted = pipe.predict(X_test)

# Model Accuracy
print("Logistic Regression Accuracy:",metrics.accuracy_score(y_test, predicted))

/home/mfalcon/projects/chequeabot/cbot/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Logistic Regression Accuracy: 0.8564705882352941


In [163]:
classifier = MultinomialNB()

pipe = Pipeline([("cleaner", predictors()),
                 ('vectorizer', CountVectorizer()),
                 ('classifier', classifier)])


# model generation
pipe.fit(X_train,y_train)

# Predicting with a test dataset
predicted = pipe.predict(X_test)

# Model Accuracy
print("Logistic Regression Accuracy:",metrics.accuracy_score(y_test, predicted))

Logistic Regression Accuracy: 0.8486274509803922


In [162]:
classifier = MultinomialNB()

pipe = Pipeline([("cleaner", predictors()),
                 ('vectorizer', CountVectorizer()),
                 ('tfidf', TfidfTransformer()),
                 ('classifier', classifier)])


# model generation
pipe.fit(X_train,y_train)

# Predicting with a test dataset
predicted = pipe.predict(X_test)

# Model Accuracy
print("Logistic Regression Accuracy:",metrics.accuracy_score(y_test, predicted))

Logistic Regression Accuracy: 0.8243137254901961


In [161]:
from sklearn.linear_model import SGDClassifier

classifier = SGDClassifier(loss='hinge', penalty='l2',
                           alpha=1e-3, random_state=42,
                           max_iter=5, tol=None)

pipe = Pipeline([("cleaner", predictors()),
                 ('vectorizer', CountVectorizer()),
                 ('tfidf', TfidfTransformer()),
                 ('classifier', classifier)])


# model generation
pipe.fit(X_train,y_train)

# Predicting with a test dataset
predicted = pipe.predict(X_test)

# Model Accuracy
print("Logistic Regression Accuracy:",metrics.accuracy_score(y_test, predicted))


Logistic Regression Accuracy: 0.8250980392156863
